In [ ]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [ ]:
app = dash.Dash()

In [ ]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [ ]:
conapo = pd.read_csv('conapo.csv')

In [ ]:
servicios = pd.read_csv('../BD-sucias/servicios_18_OCTUBRE_2021.csv', low_memory=False)

In [ ]:
victimas = pd.read_csv('reportes/reporte_semujeres_18_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [ ]:
subtipo = pd.read_csv('../BD-sucias/subtipo_18_OCTUBRE_2021.csv', low_memory=False)
subtipo.fecha_recepcion = pd.to_datetime(subtipo.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
subtipo.fecha_hechos = pd.to_datetime(subtipo.fecha_hechos, format='%d/%m/%Y', errors='ignore')
subtipo = subtipo.dropna()
selecciones = subtipo[subtipo.SubtipoOrd=='Seleccione'].index
subtipo.loc[selecciones, 'SubtipoOrd'] = 'No especificado'

In [ ]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
victimas['Feminicida']=0
fems = victimas[victimas['descripcion_otro_tipos']=='FEMINICIDA'].index
victimas.loc[fems,'Feminicida'] = 1
victimas.loc[fems,'Otro'] = 0

In [ ]:
feminicidios = pd.read_csv('../BD-sucias/feminicidios-corregida.csv', low_memory=False)
feminicidios.fecha_recepcion = pd.to_datetime(feminicidios.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
feminicidios['año_recepcion'] = feminicidios['fecha_recepcion'].dt.year
nas = feminicidios[(feminicidios.TipoRelacion.isna()==True)|(feminicidios.TipoRelacion=='Seleccione')].index
feminicidios.loc[nas,'TipoRelacion'] = 'Desconocido'
com = feminicidios[(feminicidios.TipoRelacion=='En la comunidad')].index
feminicidios.loc[com,'TipoRelacion'] = 'Comunidad'

In [ ]:
feminicidios = pd.concat([feminicidios, feminicidios.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
feminicidios['Feminicida']=0
fems = feminicidios[feminicidios['descripcion_tipo_violencia'].str.contains('FEM')].index
feminicidios.loc[fems,'Feminicida'] = 1
feminicidios.loc[fems,'Otro'] = 0

In [ ]:
import re
def mayusculas(s):
    return ' '.join([i.strip() for i in re.findall('[A-Z][^A-Z]*', s)])
def numeros(n):
    return ', '.join(n.split('.0'))
def tokenize(s):
    temp = s.sub(',','')
    return ', '.join(temp)
def unir_unicos(s):
    return ', '.join(set(s))
def unir_resto(s):
    return ', '.join(set(s.split(' ')))

In [ ]:
app.layout = html.Div(
    children=[
        html.H1(children="INDICADORES MUNICIPALES TABLERO",),
        html.P(
            children="Texto de prueba",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=victimas.fecha_recepcion.min(),
                            max_date_allowed=victimas.fecha_recepcion.max(),
                            start_date=victimas.fecha_recepcion.min(),
                            end_date=victimas.fecha_recepcion.max(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Tasa de violencia municipal (año)",
                            className="menu-title"
                            ),
                        dcc.Input(
                            id="tasa",
                            type='number',
                            placeholder="Año",
                            value=2020,
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        dcc.Graph(
            id = 'map_tasa',  
        ),
        dcc.Graph(
            id = 'fig_1',  
        ),
        dcc.Graph(
            id = 'map_1',  
        ),
    ]
)

In [ ]:
feminicidios_copia = feminicidios

In [ ]:
conapo_copia=conapo

In [ ]:
@app.callback( 
    Output('map_tasa', "figure"),
    Input("tasa", "value"), 
)
def update_tasa(year):
    mask = (
        (victimas.año_recepcion == year)
    )
    tasa_mask = (victimas.año_recepcion == year)

    filtered_data = victimas.loc[mask, :]
    filtered_tasa = victimas.loc[tasa_mask, :]

    # conapo year
    pob_fem = (conapo_copia[(conapo_copia['CLAVE_ENT']==31)&(conapo_copia['AÑO']==year)&(conapo_copia['SEXO']=='Mujeres')]).reset_index(drop=True)
    pob_fem = pob_fem[['MUN','POB']]
    pob_fem = pob_fem.groupby('MUN').sum()

    # Tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    
    # Mapa tasas violencias
    victimas_2020 = filtered_tasa
    casos_mun = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
    casos_mun = yuc[['geometry_mun','nom_mun']].merge(casos_mun, right_on=casos_mun.index, left_on='nom_mun', how='left')
    casos_mun.fillna(0,inplace=True)
    casos_mun = pd.DataFrame(casos_mun.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))
    casos_mun[['fk_euv','POB']] = casos_mun[['fk_euv','POB']].astype(int)

    mun_violencias = victimas_2020.groupby('municipiohechos')[['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']].sum()
    mun_violencias = yuc[['geometry_mun','nom_mun']].merge(mun_violencias, right_on=mun_violencias.index, left_on='nom_mun', how='left')
    mun_violencias.fillna(0, inplace=True)

    mun_violencias = pd.DataFrame(mun_violencias.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))
    for violencia in ['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']:
        mun_violencias['{} Tasa 1000'.format(violencia)] = round(mun_violencias[violencia]/mun_violencias['POB'].astype(int)*1000,2)
        #mun_violencias['{}_text'.format(violencia)] = mun_violencias.apply(lambda row: f"Casos: {int(row[violencia])}<br>POBFEM: {int(row['POBFEM'])}", axis=1)
        mun_violencias['{}_casos'.format(violencia)] = mun_violencias.apply(lambda row: int(row[violencia]), axis=1)

    mun_violencias = mun_violencias.merge(casos_mun[['fk_euv','nom_mun']], left_on=mun_violencias.index, right_on=casos_mun.index, how='left')
    mun_violencias[['fk_euv','POB']] = mun_violencias[['fk_euv','POB']].astype(int)
    mun_violencias['Tasa 1000 Total'] =round(mun_violencias.fk_euv/mun_violencias.POB*1000,2)
    mun_violencias.drop(['key_0','nom_mun_y'],axis='columns', inplace=True)
    #mun_violencias['text_total'] = mun_violencias.apply(lambda row: f"Casos Totales: {int(row['fk_euv'])}<br>POBFEM: {row['POBFEM']}", axis=1)

    gdf = geopandas.GeoDataFrame(mun_violencias, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun_x')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

    cols_dd = ['Tasa 1000 Total','Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro',]
    # we need to add this to select which trace 
    # is going to be visible
    visible = np.array(cols_dd)

    poly_json = json.loads(gdf.geometry_mun.to_json())

    # define traces and buttons at once
    traces = []
    buttons = []

    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['Tasa 1000 Total'], # Data to be color-coded
            colorbar_title='Tasa',
            name='Tasa {}'.format(year),
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf.index, # hover text
            customdata=np.stack([ gdf['POB'], gdf['fk_euv']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos Totales: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
            visible= True if 'Tasa 1000 Total'==cols_dd[0] else False,
            ))

    buttons.append(dict(label='Tasa 1000 Total',
                        method="update",
                        args=[{"visible":list(visible=='Tasa 1000 Total')},
                                {"title":f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres"}]))

    # Mapas de tipos de violencia
    for value in cols_dd[1:]:
        traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['{} Tasa 1000'.format(value)], # Data to be color-coded
            colorbar_title='Tasa',
            name='Tasa de violencia<br>'+value,
            #hoverinfo='{}+{} Tasa 1000'.format(value,value),
            #hovertemplate = "Mun: %{x}".format(gdf.index),
            #text = gdf['{}_text'.format(value)], # hover text
            visible= True if value==cols_dd[0] else False,
            marker_line_width=1,
            colorscale="Viridis",
            text = gdf.index, # hover text
            customdata=np.stack([ gdf['POB'], gdf['{}_casos'.format(value)]], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
            #visible= False,
            ))
                                        

        buttons.append(dict(label=value,
                            method="update",
                            args=[{"visible":list(visible==value)},
                                {"title":f"Mapa 1. Tasa de casos de violencia {value} por cada 1000 mujeres"}]))


    updatemenus = [{"active":0,
                    "buttons":buttons,
                }]


    # Show figure
    map_tasa = go.Figure(data=traces,
                    layout=dict(updatemenus=updatemenus))
    # This is in order to get the first title displayed correctly
    first_title = cols_dd[0]
    map_tasa.update_layout(title=f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres",)
    map_tasa.update_geos(fitbounds="locations", visible=False, )
    map_tasa.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(victimas_2020.año_recepcion),len(victimas_2020)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_tasa.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})

    return map_tasa

In [ ]:
@app.callback(
    [   
        #Output('map_tasa', "figure"),
        Output('fig_1', "figure"),
        Output('map_1', "figure"),
    ],    
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
        #Input("tasa", "value"),
    ]
)
def update_charts(start_date, end_date):
    mask = (
        (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )
    feminicidios = feminicidios_copia
    fem_mask = (
        (feminicidios.fecha_recepcion >= start_date)
        & (feminicidios.fecha_recepcion <= end_date)
    )
    #tasa_mask = (victimas.año_recepcion == year)

    filtered_data = victimas.loc[mask, :]
    filtered_fems = feminicidios.loc[fem_mask, :]
    #filtered_tasa = victimas.loc[tasa_mask, :]

    # conapo year
    """pob_fem = (conapo_copia[(conapo_copia['CLAVE_ENT']==31)&(conapo_copia['AÑO']==year)&(conapo_copia['SEXO']=='Mujeres')]).reset_index(drop=True)
    pob_fem = pob_fem[['MUN','POB']]
    pob_fem = pob_fem.groupby('MUN').sum()"""

    # Tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    
    """# Mapa tasas violencias
    victimas_2020 = filtered_tasa
    casos_mun = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
    casos_mun = yuc[['geometry_mun','nom_mun']].merge(casos_mun, right_on=casos_mun.index, left_on='nom_mun', how='left')
    casos_mun.fillna(0,inplace=True)
    casos_mun = pd.DataFrame(casos_mun.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))
    casos_mun[['fk_euv','POB']] = casos_mun[['fk_euv','POB']].astype(int)

    mun_violencias = victimas_2020.groupby('municipiohechos')[['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']].sum()
    mun_violencias = yuc[['geometry_mun','nom_mun']].merge(mun_violencias, right_on=mun_violencias.index, left_on='nom_mun', how='left')
    mun_violencias.fillna(0, inplace=True)

    mun_violencias = pd.DataFrame(mun_violencias.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))
    for violencia in ['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']:
        mun_violencias['{} Tasa 1000'.format(violencia)] = round(mun_violencias[violencia]/mun_violencias['POB'].astype(int)*1000,2)
        #mun_violencias['{}_text'.format(violencia)] = mun_violencias.apply(lambda row: f"Casos: {int(row[violencia])}<br>POBFEM: {int(row['POBFEM'])}", axis=1)
        mun_violencias['{}_casos'.format(violencia)] = mun_violencias.apply(lambda row: int(row[violencia]), axis=1)

    mun_violencias = mun_violencias.merge(casos_mun[['fk_euv','nom_mun']], left_on=mun_violencias.index, right_on=casos_mun.index, how='left')
    mun_violencias[['fk_euv','POB']] = mun_violencias[['fk_euv','POB']].astype(int)
    mun_violencias['Tasa 1000 Total'] =round(mun_violencias.fk_euv/mun_violencias.POB*1000,2)
    mun_violencias.drop(['key_0','nom_mun_y'],axis='columns', inplace=True)
    #mun_violencias['text_total'] = mun_violencias.apply(lambda row: f"Casos Totales: {int(row['fk_euv'])}<br>POBFEM: {row['POBFEM']}", axis=1)

    gdf = geopandas.GeoDataFrame(mun_violencias, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun_x')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

    cols_dd = ['Tasa 1000 Total','Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro',]
    # we need to add this to select which trace 
    # is going to be visible
    visible = np.array(cols_dd)

    poly_json = json.loads(gdf.geometry_mun.to_json())

    # define traces and buttons at once
    traces = []
    buttons = []

    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['Tasa 1000 Total'], # Data to be color-coded
            colorbar_title='Tasa',
            name='Tasa {}'.format(year),
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf.index, # hover text
            customdata=np.stack([ gdf['POB'], gdf['fk_euv']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos Totales: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
            visible= True if 'Tasa 1000 Total'==cols_dd[0] else False,
            ))

    buttons.append(dict(label='Tasa 1000 Total',
                        method="update",
                        args=[{"visible":list(visible=='Tasa 1000 Total')},
                                {"title":f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres"}]))

    # Mapas de tipos de violencia
    for value in cols_dd[1:]:
        traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['{} Tasa 1000'.format(value)], # Data to be color-coded
            colorbar_title='Tasa',
            name='Tasa de violencia<br>'+value,
            #hoverinfo='{}+{} Tasa 1000'.format(value,value),
            #hovertemplate = "Mun: %{x}".format(gdf.index),
            #text = gdf['{}_text'.format(value)], # hover text
            visible= True if value==cols_dd[0] else False,
            marker_line_width=1,
            colorscale="Viridis",
            text = gdf.index, # hover text
            customdata=np.stack([ gdf['POB'], gdf['{}_casos'.format(value)]], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
            #visible= False,
            ))
                                        

        buttons.append(dict(label=value,
                            method="update",
                            args=[{"visible":list(visible==value)},
                                {"title":f"Mapa 1. Tasa de casos de violencia {value} por cada 1000 mujeres"}]))


    updatemenus = [{"active":0,
                    "buttons":buttons,
                }]


    # Show figure
    map_tasa = go.Figure(data=traces,
                    layout=dict(updatemenus=updatemenus))
    # This is in order to get the first title displayed correctly
    first_title = cols_dd[0]
    map_tasa.update_layout(title=f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres",)
    map_tasa.update_geos(fitbounds="locations", visible=False, )
    map_tasa.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(victimas_2020.año_recepcion),len(victimas_2020)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_tasa.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})"""

    # fig 1 - violencias tipos
    tipos = tipos.sort_values(0, ascending=False)
    y = tipos[0].values
    y_total = len(victimas)
    fig_1 = px.bar(x=tipos.index,
                y=tipos[0],
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:7],
                color_discrete_map="identity",
                title='Gráfica 1. Casos de violencia hacia las mujeres por tipo de violencia'
                )
    fig_1.update_xaxes(type='category')
    fig_1.update_layout( xaxis_title=None,  height=500)
    fig_1.update_traces(texttemplate='%{text} %')

    # Feminicidios var
    feminicidios = filtered_fems.merge(filtered_data[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
    feminicidios = feminicidios.drop_duplicates(keep='last')
    feminicidios = feminicidios.reset_index(drop=True)

    # Mapa violencias
    feminicidios_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==False]
    tentativas_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==True]
    fem_mun = feminicidios_sent.municipioV.value_counts()
    fem_mun = yuc[['geometry_mun','nom_mun']].merge(fem_mun, right_on=fem_mun.index, left_on='nom_mun', how='left')
    fem_mun.fillna(0,inplace=True)
    fem_mun.municipioV = fem_mun.municipioV.astype(int)

    fem_mun_tent = tentativas_sent.municipioV.value_counts()
    fem_mun_tent = yuc[['geometry_mun','nom_mun']].merge(fem_mun_tent, right_on=fem_mun_tent.index, left_on='nom_mun', how='left')
    fem_mun_tent.fillna(0,inplace=True)
    fem_mun_tent.municipioV = fem_mun_tent.municipioV.astype(int)

    merged = fem_mun.merge(feminicidios_sent[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
    #merged.fillna(0,inplace=True)
    merged = merged.replace(np.nan, '').astype(str)
    merged[merged.notnull()] = merged[merged.notnull()].astype(object)
    #merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
    merged.drop(columns='municipioV_y',inplace=True)
    merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

    merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
    merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
    merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
    merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
    merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
    merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
    merged = merged.replace(',','', regex=True)


    merged.tipos = merged.tipos.str.split()
    merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
    merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
    merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
    merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
    merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
    merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

    test = merged.EdadVictima.apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['EdadVictima'] = round(test.values/merged.len)

    test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['Edad Agresor'] = round(test.values/merged.len)

    merged = merged.replace(np.nan, '').astype(str)
    merged = merged.replace('0.0', 'Sin datos').astype(str)
    #merged['EdadVictima'].values
    #merged.to_csv('test.csv', encoding='utf-8-sig')

    fem_mun = fem_mun.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
    fem_mun.fillna('', inplace=True)

    #fem_mun = fem_mun.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

    fem_mun['bins']=pd.cut(fem_mun["municipioV"],
        bins=[-1, 0.5, 1.5, 2.5,np.inf], 
        labels=['0','1','2','3 o más'])

    gdf = geopandas.GeoDataFrame(fem_mun, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

    merged = fem_mun_tent.merge(tentativas_sent[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
    #merged.fillna(0,inplace=True)
    merged = merged.replace(np.nan, '').astype(str)
    merged[merged.notnull()] = merged[merged.notnull()].astype(object)
    #merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
    merged.drop(columns='municipioV_y',inplace=True)
    merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

    merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
    merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
    merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
    merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
    merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
    merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
    merged = merged.replace(',','', regex=True)


    merged.tipos = merged.tipos.str.split()
    merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
    merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
    merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
    merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
    merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
    merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

    test = merged.EdadVictima.apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['EdadVictima'] = round(test.values/merged.len)

    test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['Edad Agresor'] = round(test.values/merged.len)

    merged = merged.replace(np.nan, '').astype(str)
    merged = merged.replace('0.0', 'Sin datos').astype(str)
    #merged['EdadVictima'].values
    #merged.to_csv('test.csv', encoding='utf-8-sig')

    fem_mun_tent = fem_mun_tent.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
    fem_mun_tent.fillna('', inplace=True)

    #fem_mun_tent =fem_mun_tent.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

    fem_mun_tent['bins']=pd.cut(fem_mun_tent["municipioV"],
        bins=[-1, 0.5, 1.5, 2.5,np.inf], 
        labels=['0','1','2','2 o más'])

    gdf_tent = geopandas.GeoDataFrame(fem_mun_tent, geometry='geometry_mun')
    gdf_tent = gdf_tent.set_index('nom_mun')
    gdf_tent = gdf_tent.to_crs(pyproj.CRS.from_epsg(4326))

    # colores
    colors = px.colors.qualitative.Prism[4:8]
    
    cols_dd = ['Feminicidios','Tentativas']
    # we need to add this to select which trace 
    # is going to be visible
    visible = np.array(cols_dd)

    poly_json = json.loads(gdf.geometry_mun.to_json())

    # define traces and buttons at once
    traces = []
    buttons = []

    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['municipioV'], # Data to be color-coded
            colorscale=[[0, colors[0]],[0.25, colors[0]],
            [0.25, colors[1]],[0.5, colors[1]],
            [0.5, colors[2]],[0.75, colors[2]],
            [0.75, colors[3]], [1, colors[3]],],
            zmin=0, zmax=3,
            colorbar=dict(
            title="Núm. Víctimas<br>",
            titleside="top",
            tickmode="array",
            tickvals=[0.375, 1.125, 1.875, 2.625],
            ticktext=["0", "1", "2", "3 o más"],
            ticks="outside"
            ),
            #colorbar_title='Casos',
            name='Feminicidios',
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf.index, # hover text
            customdata=np.stack([gdf['TipoRelacion'], gdf['EdadVictima'], gdf['Edad Agresor'], gdf['tipos'], gdf['año_recepcion'], gdf['TipoRelacion']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Número de Víctimas: ' + '%{z}' +  '<br>' +'Promedio Edad Víctimas: ' 
            + '%{customdata[1]}' +'<br>Promedio Edad Agresores: ' + '%{customdata[2]}'+'<br>Tipos de violencia: ' + '%{customdata[3]}'
            +'<br>Año de recepción del caso: ' + '%{customdata[4]}' + '<br>Vínculo con el agresor: ' + '%{customdata[5]}',
            visible= True
            ))

    buttons.append(dict(label='Feminicidios',
                        method="update",
                        args=[{"visible":list(visible=='Feminicidios')},
                                {"title":f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio"}]))

    poly_json = json.loads(gdf_tent.geometry_mun.to_json())
    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf_tent.index, # Spatial coordinates
            z=gdf_tent['municipioV'], # Data to be color-coded
            colorscale=[[0, colors[0]],[0.25, colors[0]],
            [0.25, colors[1]],[0.5, colors[1]],
            [0.5, colors[2]],[0.75, colors[2]],
            [0.75, colors[3]], [1, colors[3]],],
            zmin=0, zmax=3,
            colorbar=dict(
            title="Núm. Víctimas<br>",
            titleside="top",
            tickmode="array",
            tickvals=[0.375, 1.125, 1.875, 2.625],
            ticktext=["0", "1", "2", "3 o más"],
            ticks="outside"
            ),
            #colorbar_title='Casos',
            name='Tentativas',
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf_tent.index, # hover text
            customdata=np.stack([gdf_tent['TipoRelacion'], gdf_tent['EdadVictima'], gdf_tent['Edad Agresor'], gdf_tent['tipos'],gdf_tent['año_recepcion'], gdf_tent['TipoRelacion']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Número de Víctimas: ' + '%{z}' +  '<br>' +'Promedio Edad Víctimas: ' 
            + '%{customdata[1]}' +'<br>Promedio Edad Agresores: ' + '%{customdata[2]}'+'<br>Tipos de violencia: ' + '%{customdata[3]}'
            +'<br>Año de recepción del caso: ' + '%{customdata[4]}' + '<br>Vínculo con el agresor: ' + '%{customdata[5]}',
            visible= False,
            ))

    buttons.append(dict(label='Tentativas',
                        method="update",
                        args=[{"visible":list(visible=='Tentativas')},
                                {"title":f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio"}]))


    updatemenus = [{"active":0,
                    "buttons":buttons,
                }]


    # Show figure
    fig = go.Figure(data=traces,
                    layout=dict(updatemenus=updatemenus))
    # This is in order to get the first title displayed correctly
    #first_title = cols_dd[0]
    fig.update_layout(title=f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio",)
    fig.update_geos(fitbounds="locations", visible=False, )
    fig.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Femincidios:</b> {}<br><b>Tentativas:</b> {}".format(min(feminicidios.año_recepcion),max(feminicidios.año_recepcion),(len(feminicidios_sent)),(len(tentativas_sent))),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    fig.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})

    return fig_1, fig

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)